<a href="https://colab.research.google.com/github/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/blob/main/Machine_Learning_NLP_PreProcesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO


Los emojis los traduce bien pero no como queremos, nos da igual que nos diga face_with_tears_of_joy, el objetivo es que tears y joy se integren con el texto, eso hay que revisarlo.

No sé hasta que punto nos interesa mantener los hastag




In [17]:
# dataframes library
!pip install pyspellchecker 
!pip install contractions
!pip install nltk --upgrade
!pip install emoji

import pandas as pd
import re
import nltk
import sys
import csv
nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter not necessary

from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

#Fixes and emoji
import emoji
from textblob import TextBlob
from spellchecker import SpellChecker
import contractions
from multiprocessing import Pool
from collections import Counter

#Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

#Feature selection 
from sklearn.feature_selection import SelectKBest, chi2

#Classification
from sklearn.model_selection import train_test_split

from sklearn import neighbors
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels



Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [18]:
english_stops = set(stopwords.words('english'))
spell = SpellChecker(distance=1)

#We add the frequent words used in tweets that do not appear in our dictionary but are importants.
spell.word_frequency.load_words(['nigga', 'niggas', 'retweet', 'eggplant',
                                 'oreo' ,'gf', 'lmao', 'emoji', 'skippy',
                                 'piercings', 'gaywad','gay-wad','smh','bruh',
                                 'pornhub', 'gtfo', 'mixtape', 'wtf','lmfao',
                                 'twerking','tf','twerk','cowgirls','stfu',
                                 'fohead', 'thots', 'thot', 'friendzone',
                                 'scally', 'bestfriend', 'cuz', 'motherfucker',
                                 'motherfuckers', 'youtube', 'murda', 'idk'])
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
unknown_arr = []
misspelled_arr = []


In [21]:
raw_dataset_tweets = pd.read_csv('https://raw.githubusercontent.com/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/main/labeled_data.csv?token=AGI3DY5237W3OU7AOBOOER276Y2X6', index_col=[0])

# Preprocesing:

Mandatory preprocessing steps
 - Remove unseful data: ! "_ $% & / ( ) = _ ˆ* ¡@
 - Remove all capital letters
 - Correct wrong words: https://norvig.com/spell-correct.html
 - Lemmatize all terms

Optional preprocessing steps:
 - Remove contractions: don’t -> do not
 - Remove repeated words: great great show
 - Replace emoticons, for example,🙂 with “smile”, 😉with “ok”


We just need index and tweet content for this first part, so we drop Crowdflowers users information from the dataset.

In [22]:
dataset_tweets_preprocesing = raw_dataset_tweets.copy()
dataset_tweets_preprocesing.drop("count", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("hate_speech", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("offensive_language", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("neither", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("class", axis = 1, inplace = True)

In [23]:
dataset_tweets_preprocesing

,tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...
25291,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292,"you've gone and broke the wrong heart baby, an..."
25294,young buck wanna eat!!.. dat nigguh like I ain...
25295,youu got wild bitches tellin you lies


In [24]:
#@title Number of tweets
num_tweets = 24783 #@param {type:"slider", min:1, max:24783, step:10}

examples_preprocesing = dataset_tweets_preprocesing.iloc[0:num_tweets,0]
examples_preprocesing.reset_index(inplace = True, drop= True)
print(examples_preprocesing)

lista_tweets = []



0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: tweet, Length: 24783, dtype: object


### Definimos las funciones que procesarán los tweets


In [25]:

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in english_stops])

def stopwords_tokens(arr):
  text_arr = []
  for token in arr:
    if token not in english_stops:
      text_arr.append(token)
  return text_arr

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def demojizar(arr):
  text_arr = []
  trozos_emoji = []
  for token in arr:
    wordemoji = emoji.demojize(token, delimiters=("", ""))
    trozos_emoji = wordemoji.split('_')
    for word in trozos_emoji:
      text_arr.append(word)
  return text_arr

def textConditions(token):
  if re.match("[`!\"_$%&/()=_ˆ*¡,@:.]+",token): #Punctuation marks
    return False
  if re.match("[-><~]+",token): #Punctuation marks
    return False
  if re.match("[^\u0000-\u007F]+",token): #Unicode
    return False
  return True

def processArray(arr):
  text_arr = []
  arr = list(set(arr))
  for token in arr:
    if textConditions(token):
      token = stemmer.stem(token)
      token = lemmatizer.lemmatize(token) # < ---- LEMMA
      text_arr.append(token)
  return text_arr

def correccion_soft(arr):

  correct_arr = []

  correct = spell.known(arr)
  misspelled = spell.unknown(arr)

  for token in correct:
    correct_arr.append(token)

  for token in misspelled:
    token = replaceThreeOrMore(token)
    candidate = spell.correction(token)
    correct_arr.append(candidate)
  return correct_arr

def replaceThreeOrMore(text):
  pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
  return pattern.sub(r"\1\1", text)

def correccion_hard(arr):

  correct_arr = []

  correct = spell.known(arr)
  misspelled = spell.unknown(arr)

  for token in correct:
    correct_arr.append(token)

  for token in misspelled:
      misspelled_arr.append(token)
      token = replaceThreeOrMore(token)
      candidate = spell.correction(token)
      if candidate == token:
        unknown_arr.append(token)
      else:
        correct_arr.append(candidate)

  return correct_arr



In [26]:
ds_preproc_hard = examples_preprocesing.copy()
ds_preproc_hard = ds_preproc_hard.str.lower() #lower case
ds_preproc_hard = ds_preproc_hard.apply(contractions.fix) #repair contractions

ds_preproc_hard = ds_preproc_hard.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remove users
ds_preproc_hard = ds_preproc_hard.str.replace('\”\“\’[`!\"_$%/()=_ˆ*¡,@:.]+','') #Remove unseful data
ds_preproc_hard = ds_preproc_hard.str.replace('rt','') #Remove rt's


ds_preproc_hard = ds_preproc_hard.apply(stopwords) #Remove Stopwords
ds_preproc_hard = ds_preproc_hard.apply(remove_urls) #Remove url

ds_preproc_hard = ds_preproc_hard.apply(TweetTokenizer().tokenize) #Tokenizar
ds_preproc_hard = ds_preproc_hard.apply(demojizar) #Traducir emoji a texto

ds_preproc_soft = ds_preproc_hard.copy()

ds_preproc_hard = ds_preproc_hard.apply(correccion_hard)


ds_preproc_hard = ds_preproc_hard.apply(stopwords_tokens)
ds_preproc_hard = ds_preproc_hard.apply(processArray) #Lemma and repeated words hard

ds_preproc_soft = ds_preproc_soft.apply(correccion_soft)
ds_preproc_soft = ds_preproc_soft.apply(stopwords_tokens)
ds_preproc_soft = ds_preproc_hard.apply(processArray) #Lemma and repeated words soft


In [27]:
unknown_arr = list(set(unknown_arr))
print(unknown_arr)

['#barrybonds', '5:30', 'lewisville', '#sickipedia', 'ratchetest', '#ass2fat', 'reblog', 'quisiera', 'pappadeaux', '#pattyfuckingboges', 'mckean', 'maryjane', '#tedgibson', '1-800-', '#idgafmode', 'hoofdstuk', 'gwbush', 'honky-tonk', '#merched', '#teamignant', '18th', 'sidechick', '#fuckwhatyouheard', '#getatme', 'midlaner', 'f-of-r-d', 'wesgod', '#mainestatepier', 'democr', 'club.they', 'sirhampton', 'lmaosjdusubejs', 'fancily', '#michiganst', '#turnthatshituplouder', 'oharrd', '#mentionadislike', 'janoskians', '#selfiesonselfies', 'soo', '8-)', '10mins', '#selectionsunday', 'vacarro', '#pleaseoptionthistweetnow', 'ongesteld', '#spamblocked', '#100', '#mngop', '#withthesebitches', '#smokeweed', '#doyourgoogles', '#lookingformore', '#littledebbie', 'crazzyy', 'spunge-cake', 'clorox', '#vmas', '6-0', "#wcw's", 'midsole', 'out-of-body', '#bbw', 'txtn', 'tokuda', 'kimlau', 'playcalling', '#powercouple', 'manbooty', 'come-at-me-bro', 'idfwu', 'monn-neyy', 'pimpinkendotnet', 'bitvj', '/-:',

In [28]:
Counter(unknown_arr)

Counter({'#barrybonds': 1,
         '5:30': 1,
         'lewisville': 1,
         '#sickipedia': 1,
         'ratchetest': 1,
         '#ass2fat': 1,
         'reblog': 1,
         'quisiera': 1,
         'pappadeaux': 1,
         '#pattyfuckingboges': 1,
         'mckean': 1,
         'maryjane': 1,
         '#tedgibson': 1,
         '1-800-': 1,
         '#idgafmode': 1,
         'hoofdstuk': 1,
         'gwbush': 1,
         'honky-tonk': 1,
         '#merched': 1,
         '#teamignant': 1,
         '18th': 1,
         'sidechick': 1,
         '#fuckwhatyouheard': 1,
         '#getatme': 1,
         'midlaner': 1,
         'f-of-r-d': 1,
         'wesgod': 1,
         '#mainestatepier': 1,
         'democr': 1,
         'club.they': 1,
         'sirhampton': 1,
         'lmaosjdusubejs': 1,
         'fancily': 1,
         '#michiganst': 1,
         '#turnthatshituplouder': 1,
         'oharrd': 1,
         '#mentionadislike': 1,
         'janoskians': 1,
         '#selfiesonselfies

In [29]:
mispelled_arr = list(set(misspelled_arr))
print(misspelled_arr)

['...', '...', 'dwn', '1st', 'cuffin', 'tyga', '\ue011', '..', '..', '”', '“', 'mariam', 'keeks', '...', 'plz', 'b', 'dr', 'txt', '#shots', '#sevenone', '#2mw', '#happyhumpday', 'b', '”', '...', '“', 'teanna', '...', 'n', '...', '..', '’', 't', 'simplyaddictedtoguys', 'lemmie', 'see-no-evil', 'gt', 'cuain', '...', '#ahmesehwetness', '#eagles', '#eaglesnation', 'bathtime', 'awwwwe', 'bs', 'b', 'mvps', 'funnnnnnn', 'ppl', '...', '...', '..', 'cuteeeee', 'buttcheek', 'k', 'fr', 'lulwhore', '...', 'alsarabsss', 'bihday', '..', 'make-up', '#earlychristmas', 'prople', 'richnow', 'lmaooooooooooo', '..', 'foh', 'plz', 'g', 'lbvs', 'cuzzo', 'foh', 'bc', 'wayyyyy', 'idc', '#stopwhitepeople2014', '...', '#cowboysnation', '12th', '#tehgodclan', ':p', 'dm', "read'them", '#yankees', 'staed', '..', 'yoooooouuuu', 'wheet', '...', 'cc', '#freemoneymelle', 'p.of', 'screamn', '..', 'lmmfao', '“', 'ainna', 'n', 'speak-no-evil', 'preciate', 'bday', 'wcw', '...', '...', 'idc', '...', 'fr', '#oomf', 'caoon',

In [30]:
Counter(misspelled_arr)

Counter({'...': 1892,
         'dwn': 2,
         '1st': 37,
         'cuffin': 10,
         'tyga': 16,
         '\ue011': 1,
         '..': 919,
         '”': 1245,
         '“': 1564,
         'mariam': 2,
         'keeks': 1,
         'plz': 11,
         'b': 146,
         'dr': 12,
         'txt': 4,
         '#shots': 1,
         '#sevenone': 1,
         '#2mw': 1,
         '#happyhumpday': 1,
         'teanna': 1,
         'n': 252,
         '’': 218,
         't': 102,
         'simplyaddictedtoguys': 1,
         'lemmie': 1,
         'see-no-evil': 32,
         'gt': 9,
         'cuain': 1,
         '#ahmesehwetness': 1,
         '#eagles': 3,
         '#eaglesnation': 3,
         'bathtime': 1,
         'awwwwe': 1,
         'bs': 13,
         'mvps': 1,
         'funnnnnnn': 1,
         'ppl': 59,
         'cuteeeee': 1,
         'buttcheek': 1,
         'k': 44,
         'fr': 17,
         'lulwhore': 1,
         'alsarabsss': 1,
         'bihday': 80,
         'make-up': 4

In [31]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ds_preproc_hard.to_csv(r'/content/gdrive/My Drive/preprocesado_hard.csv')
ds_preproc_soft.to_csv(r'/content/gdrive/My Drive/preprocesado_soft.csv')

Mounted at /content/gdrive
